# 🧬 Simulación y Análisis de Alanina Dipéptido - Tutorial Completo

## Introducción Detallada

### ¿Qué es la Alanina Dipéptido?

La **alanina dipéptido** (también llamada Ace-Ala-Nme) es una molécula pequeña compuesta por:
- **Ace**: Grupo acetilo (CH₃-CO-) en el extremo N-terminal
- **Ala**: Un residuo de alanina (aminoácido más simple con cadena lateral -CH₃)
- **Nme**: Grupo N-metil amida (-NH-CH₃) en el extremo C-terminal

**Total**: ~22 átomos solamente

### ¿Por qué es Importante para Aprender MD?

1. **Sistema pequeño = simulaciones rápidas**
   - En tu computadora personal: minutos, no días
   - Ideal para aprender sin esperar

2. **Muestra comportamientos complejos a pesar de su tamaño**
   - Puede adoptar conformaciones tipo α-hélice
   - Puede adoptar conformaciones tipo β-sheet
   - Transiciones entre conformaciones

3. **Bien estudiado en la literatura**
   - Hay datos experimentales para comparar
   - Es un "benchmark" estándar en el campo

4. **Introduce conceptos clave**
   - Ángulos diedros (φ, ψ)
   - Diagrama de Ramachandran
   - Espacio conformacional

### Objetivos Específicos de Este Tutorial:

| Fase | Objetivo | Aprenderemos |
|------|----------|-------------|
| **Setup** | Descargar y preparar | Manejo de archivos PDB, topologías |
| **Simulación** | Ejecutar MD | Campos de fuerza, integradores, termodinámica |
| **Análisis 1** | Estabilidad | RMSD, convergencia, energías |
| **Análisis 2** | Conformaciones | Ángulos diedros, Ramachandran |
| **Análisis 3** | Movimientos | PCA, modos principales |
| **Visualización** | Ver resultados | Gráficos 2D/3D, interpretación |

### ¿Qué NO haremos (pero podrías hacer después)?

- Solvente explícito (usaremos implícito para simplicidad)
- Simulaciones largas (solo 20 ps, lo suficiente para aprender)
- Free energy calculations (tema avanzado)
- Múltiples réplicas (solo 1 simulación)

---

**Tiempo estimado**: 30-45 minutos (simulación + análisis)

**Prerequisitos**:
- OpenMM instalado (`conda activate openmm-env`)
- Conexión a internet (para descargar PDB)
- Paciencia y curiosidad 🧪

---

## 📦 Paso 1: Importar Librerías - Explicación Exhaustiva

### Filosofía: ¿Por qué tantas librerías?

Una simulación molecular no es una sola operación, es un **pipeline completo**:

```
Estructura PDB → Preparación → Simulación → Trayectoria → Análisis → Resultados
```

Cada etapa requiere herramientas especializadas:

- **OpenMM**: El "motor" que simula física molecular
- **MDTraj/MDAnalysis**: Los "mecánicos" que analizan la trayectoria
- **NumPy/Pandas**: Los "matemáticos" que hacen cálculos
- **Matplotlib/Seaborn**: Los "artistas" que visualizan

Vamos a importar cada una **explicando en detalle** qué hace y por qué la necesitamos.

In [ ]:
# =============================================================================
# SECCIÓN 1: LIBRERÍAS PARA SIMULACIÓN MOLECULAR
# =============================================================================

import openmm as mm
# ¿QUÉ ES OpenMM?
# - Un motor de simulación de dinámicas moleculares de alto rendimiento
# - Desarrollado por Stanford y financiado por NIH
# - Usa GPU (CUDA/OpenCL) para acelerar cálculos
#
# ¿QUÉ HACE?
# - Calcula fuerzas entre átomos usando potenciales físicos (ecuaciones)
# - Integra las ecuaciones de movimiento de Newton (F = ma)
# - Mueve los átomos paso a paso en el tiempo
#
# ¿POR QUÉ 'as mm'?
# - Para escribir 'mm.LangevinIntegrator()' en lugar de 'openmm.LangevinIntegrator()'
# - Es una convención común en la comunidad OpenMM

from openmm import app, unit
# ¿QUÉ ES 'app'?
# - Submódulo de OpenMM con clases de "aplicación" de alto nivel
# - Incluye: PDBFile (leer PDB), ForceField (campos de fuerza), 
#           Simulation (orquestar todo), DCDReporter (guardar trayectorias)
#
# ¿QUÉ ES 'unit'?
# - Sistema de unidades de OpenMM para evitar errores
# - En física molecular usamos unidades específicas:
#   * Longitud: nanómetros (nm) o angstroms (Å)
#   * Tiempo: picosegundos (ps) o femtosegundos (fs)
#   * Temperatura: Kelvin (K)
#   * Energía: kilojoules/mol (kJ/mol) o kilocalories/mol (kcal/mol)
#
# EJEMPLO SIN UNIDADES (MAL):
#   temperature = 300  # ¿Kelvin? ¿Celsius? ¿Fahrenheit? Ambiguo!
#
# EJEMPLO CON UNIDADES (BIEN):
#   temperature = 300 * unit.kelvin  # Explícito, sin ambigüedades
#
# ¿POR QUÉ ES CRÍTICO?
# - OpenMM internamente usa nm, ps, kJ/mol
# - Si das 300 sin unidades, asume 300 nm de distancia (¡ERROR!)
# - Con unidades, OpenMM convierte automáticamente

print("✅ OpenMM importado correctamente")
print(f"   Versión: {mm.__version__}")
print(f"   Ubicación: {mm.__file__}")

In [ ]:
# =============================================================================
# SECCIÓN 2: LIBRERÍAS PARA ANÁLISIS DE TRAYECTORIAS
# =============================================================================

import mdtraj as md
# ¿QUÉ ES MDTraj?
# - Librería especializada para leer y analizar trayectorias de MD
# - Escrita en C++ con interfaz Python (muy rápida)
# - Desarrollada por Stanford (mismo grupo que OpenMM)
#
# ¿QUÉ PUEDE HACER?
# 1. Leer formatos: DCD, XTC, TRR, NetCDF, PDB, etc.
# 2. Calcular geométricos:
#    - RMSD (cuánto se desvía la estructura)
#    - Distancias entre átomos
#    - Ángulos (bond angles, dihedral angles)
#    - Radios de giro
#    - Área superficial accesible al solvente (SASA)
# 3. Manipular trayectorias:
#    - Alinear estructuras (superpose)
#    - Extraer frames específicos
#    - Convertir entre formatos
#
# ¿POR QUÉ ES RÁPIDA?
# - Usa NumPy arrays internamente
# - Operaciones vectorizadas (procesa múltiples frames a la vez)
# - Core en C++ compilado
#
# EJEMPLO DE USO:
#   traj = md.load('trajectory.dcd', top='structure.pdb')
#   rmsd = md.rmsd(traj, traj, frame=0)  # RMSD respecto al frame 0

import MDAnalysis as mda
# ¿QUÉ ES MDAnalysis?
# - Otra librería para análisis de MD (alternativa/complementaria a MDTraj)
# - Filosofía diferente: más orientada a objetos, más flexible
# - Maneja trayectorias como "universos" con "átomos" seleccionables
#
# ¿POR QUÉ USAR LAS DOS?
# - MDTraj: mejor para cálculos geométricos simples y rápidos
# - MDAnalysis: mejor para selecciones complejas y análisis custom
#
# EJEMPLO DE DIFERENCIA:
# MDTraj:   backbone = traj.topology.select('backbone')
# MDAnalysis: backbone = u.select_atoms('backbone')
#             backbone.positions  # acceso directo a coordenadas
#
# En este tutorial usaremos principalmente MDTraj (más simple)

from MDAnalysis.analysis import rms, align
# Importamos módulos específicos de MDAnalysis:
# - 'rms': para calcular RMSD de formas más sofisticadas
# - 'align': para alinear trayectorias completas
# Estos son útiles para análisis avanzados

print("✅ MDTraj y MDAnalysis importados")
print(f"   MDTraj versión: {md.__version__}")
print(f"   MDAnalysis versión: {mda.__version__}")

In [ ]:
# =============================================================================
# SECCIÓN 3: LIBRERÍAS CIENTÍFICAS FUNDAMENTALES
# =============================================================================

import numpy as np
# ¿QUÉ ES NumPy?
# - LA librería fundamental para computación científica en Python
# - "Numerical Python"
# - Proporciona arrays multidimensionales eficientes
#
# ¿POR QUÉ ES ESENCIAL?
# En MD, TODO son arrays:
# - Coordenadas: array de shape (n_frames, n_atoms, 3)
#   Ejemplo: traj.xyz → (100 frames, 22 átomos, 3 coordenadas xyz)
# - Energías: array de shape (n_frames,)
# - Distancias: array de shape (n_frames, n_pairs)
#
# ¿QUÉ OPERACIONES HAREMOS?
# - np.mean(): promedios
# - np.std(): desviación estándar
# - np.sqrt(): raíz cuadrada
# - np.degrees(): convertir radianes a grados
# - np.argmin(), np.argmax(): encontrar índices de mín/máx
#
# DIFERENCIA CLAVE:
# Lista de Python: [1, 2, 3, 4, 5]
#   operación: [x**2 for x in lista]  → lento, requiere loop
# NumPy array:    np.array([1, 2, 3, 4, 5])
#   operación: array**2                → rápido, vectorizado

import pandas as pd
# ¿QUÉ ES Pandas?
# - Librería para manejo de datos tabulares (tablas)
# - Piensa en Excel pero programático y mucho más poderoso
# - Estructura principal: DataFrame (tabla con filas y columnas nombradas)
#
# ¿PARA QUÉ LO USAREMOS?
# OpenMM guarda logs de simulación en archivos de texto:
# Step,Time,Potential Energy,Kinetic Energy,Temperature
# 0,0.0,-50.2,25.1,298.5
# 100,0.2,-51.3,24.9,299.2
#
# Con Pandas:
# df = pd.read_csv('log.txt')
# df['Temperature'].mean()  → temperatura promedio
# df.plot(x='Time', y='Potential Energy')  → gráfico automático
#
# VENTAJAS:
# - Manejo fácil de datos con etiquetas
# - Operaciones estadísticas simples
# - Integración con Matplotlib para gráficos
# - Exportar a CSV, Excel, etc.

from scipy import stats
# ¿QUÉ ES SciPy?
# - "Scientific Python" - funciones científicas avanzadas
# - Construida sobre NumPy
# - Módulos para: estadística, optimización, interpolación, álgebra lineal, etc.
#
# ¿POR QUÉ 'stats'?
# - Submódulo de estadística
# - Nos permitirá:
#   * Calcular correlaciones: stats.pearsonr(x, y)
#   * Hacer tests: stats.ttest_ind(grupo1, grupo2)
#   * Ajustar distribuciones: stats.norm.fit(datos)
#
# NOTA: En este tutorial básico no lo usaremos mucho,
#       pero es bueno tenerlo disponible para análisis avanzados

from sklearn.decomposition import PCA
# ¿QUÉ ES scikit-learn?
# - LA librería de machine learning en Python
# - Contiene algoritmos para clasificación, regresión, clustering, etc.
#
# ¿QUÉ ES PCA?
# - Principal Component Analysis (Análisis de Componentes Principales)
# - Técnica de reducción de dimensionalidad
#
# ¿POR QUÉ ES ÚTIL EN MD?
# Problema: Una trayectoria tiene MUCHAS dimensiones:
# - 22 átomos × 3 coordenadas = 66 dimensiones
# - 100 frames = 6,600 números para visualizar
# - Imposible graficar en 2D
#
# Solución con PCA:
# - Encuentra las direcciones de máximo movimiento
# - PC1 (Componente Principal 1) = dirección de mayor varianza
# - PC2 = segunda dirección (ortogonal a PC1)
# - Proyectamos 66D → 2D preservando ~80% de la información
# - Ahora SÍ podemos hacer un scatter plot
#
# INTERPRETACIÓN:
# - PC1 podría ser: "apertura/cierre de la molécula"
# - PC2 podría ser: "rotación de un grupo metilo"
# Los componentes capturan movimientos COLECTIVOS importantes

print("✅ Librerías científicas importadas")
print(f"   NumPy versión: {np.__version__}")
print(f"   Pandas versión: {pd.__version__}")

In [ ]:
# =============================================================================
# SECCIÓN 4: LIBRERÍAS PARA VISUALIZACIÓN
# =============================================================================

import matplotlib.pyplot as plt
# ¿QUÉ ES Matplotlib?
# - LA librería estándar para gráficos en Python
# - Inspirada en MATLAB (de ahí el nombre)
# - Puede crear prácticamente cualquier tipo de gráfico 2D
#
# ¿QUÉ TIPOS DE GRÁFICOS HAREMOS?
# 1. plt.plot(x, y)     → Líneas (energía vs tiempo)
# 2. plt.scatter(x, y)  → Puntos (Ramachandran)
# 3. plt.hist(datos)    → Histogramas (distribuciones)
# 4. plt.bar(x, altura) → Barras (varianza de PCA)
#
# ¿POR QUÉ 'pyplot'?
# - Es la interfaz tipo MATLAB (simple y directa)
# - Alternativa: interfaz orientada a objetos (más compleja pero más flexible)
#
# ANATOMÍA DE UN GRÁFICO:
# fig, ax = plt.subplots()  → Crear figura y ejes
# ax.plot(x, y)             → Dibujar datos
# ax.set_xlabel('Tiempo')   → Etiquetar eje x
# ax.set_ylabel('Energía')  → Etiquetar eje y  
# plt.show()                → Mostrar

import seaborn as sns
# ¿QUÉ ES Seaborn?
# - Librería de visualización construida SOBRE Matplotlib
# - Hace gráficos más bonitos con menos código
# - Especializada en visualización estadística
#
# ¿POR QUÉ USARLO SI TENEMOS MATPLOTLIB?
# Compara:
#
# Matplotlib (feo por defecto):
#   plt.scatter(x, y)
#   # Resultado: puntos azules básicos, fondo blanco
#
# Seaborn (bonito por defecto):
#   sns.scatterplot(x=x, y=y)
#   # Resultado: colores agradables, grid, estilo profesional
#
# CARACTERÍSTICAS:
# - Paletas de colores perceptualmente uniformes
# - Temas elegantes (darkgrid, whitegrid, etc.)
# - Gráficos estadísticos complejos en 1 línea
# - Integración perfecta con Pandas DataFrames

import nglview as nv
# ¿QUÉ ES NGLView?
# - Visor 3D de moléculas INTERACTIVO para Jupyter notebooks
# - Basado en NGL (WebGL molecular graphics)
# - Permite rotar, hacer zoom, cambiar representaciones
#
# ¿QUÉ PUEDE MOSTRAR?
# - Estructuras estáticas (PDB)
# - Trayectorias completas (animaciones)
# - Múltiples estilos:
#   * cartoon: α-hélices y β-sheets
#   * ball+stick: bolas y palitos
#   * licorice: palitos (sin bolas)
#   * surface: superficie molecular
#   * ribbon: cinta del backbone
#
# COMPARACIÓN CON OTROS VISORES:
# - VMD: más poderoso pero requiere instalación separada
# - PyMOL: mejor para figuras de publicación
# - NGLView: perfecto para exploración interactiva en notebooks
#
# EJEMPLO DE USO:
#   view = nv.show_mdtraj(traj)
#   view.add_representation('cartoon')
#   # Ahora puedes rotar con el mouse

print("✅ Librerías de visualización importadas")

In [ ]:
# =============================================================================
# SECCIÓN 5: CONFIGURACIÓN DE VISUALIZACIÓN
# =============================================================================

plt.style.use('seaborn-v0_8-darkgrid')
# ¿QUÉ HACE ESTO?
# - Cambia el "tema" de TODOS los gráficos de Matplotlib
# - 'seaborn-v0_8-darkgrid' significa:
#   * Estilo tipo Seaborn (elegante)
#   * Versión 0.8 (compatibilidad)
#   * darkgrid = fondo gris claro con líneas de cuadrícula
#
# ¿POR QUÉ ES IMPORTANTE?
# Sin esto: gráficos con fondo blanco, colores básicos, sin cuadrícula
# Con esto: gráficos profesionales automáticamente
#
# OTROS ESTILOS DISPONIBLES:
# - 'default': estilo básico de Matplotlib
# - 'ggplot': estilo de R/ggplot2
# - 'bmh': estilo Bayesian Methods for Hackers
# - 'fivethirtyeight': estilo del sitio web 538
# Prueba diferentes estilos cambiando este valor

sns.set_palette("husl")
# ¿QUÉ ES UNA PALETA DE COLORES?
# - Una secuencia de colores predefinida
# - Usada cuando graficas múltiples series
#
# ¿POR QUÉ 'husl'?
# - HUSL = Human-friendly HSL (Hue, Saturation, Lightness)
# - Es "perceptualmente uniforme":
#   * Los colores son igualmente distinguibles para el ojo humano
#   * No hay colores que se "pierdan" o dominen
# - Funciona bien para:
#   * Personas con daltonismo (colorblindness-friendly)
#   * Impresión en blanco y negro (contraste preservado)
#   * Proyectores (colores se mantienen distinguibles)
#
# OTRAS PALETAS:
# - 'deep': colores profundos y saturados
# - 'muted': colores apagados (menos llamativos)
# - 'bright': colores brillantes
# - 'pastel': colores pastel suaves
# - 'dark': colores oscuros
#
# CUÁNDO IMPORTA:
# Si graficas 5 líneas, usará 5 colores de la paleta automáticamente

%matplotlib inline
# ¿QUÉ ES ESTO?
# - Un "comando mágico" de IPython/Jupyter (nota el %)
# - No es código Python normal, es específico de notebooks
#
# ¿QUÉ HACE?
# - Le dice a Jupyter: "muestra los gráficos aquí en el notebook"
# - Los gráficos aparecen como imágenes PNG debajo de la celda
#
# ¿QUÉ PASARÍA SIN ESTO?
# - Los gráficos se abrirían en ventanas separadas
# - No se guardarían con el notebook
# - Sería imposible compartir notebooks con gráficos
#
# ALTERNATIVAS:
# - %matplotlib notebook → gráficos interactivos (zoom, pan)
# - %matplotlib widget → versión moderna de notebook
# Pero 'inline' es lo más común y confiable

print("✅ Configuración de visualización establecida")
print("   - Estilo: seaborn-darkgrid")
print("   - Paleta: husl")
print("   - Backend: inline")

In [ ]:
# =============================================================================
# SECCIÓN 6: UTILIDADES Y VERIFICACIÓN
# =============================================================================

import urllib.request
# ¿PARA QUÉ ES ESTO?
# - Para descargar archivos desde URLs (internet)
# - Parte de la librería estándar de Python (no requiere instalación)
#
# ¿POR QUÉ LO NECESITAMOS?
# Vamos a descargar la estructura PDB automáticamente:
# url = "https://files.rcsb.org/download/1ALA.pdb"
# urllib.request.urlretrieve(url, 'alanina.pdb')
#
# VENTAJA:
# - El notebook es completamente autónomo
# - No necesitas descargar archivos manualmente
# - Cualquiera puede ejecutarlo y obtendrá los mismos resultados
#
# ALTERNATIVA:
# Podrías usar: requests.get(url).content
# Pero urllib es más simple para archivos únicos

import warnings
warnings.filterwarnings('ignore')
# ¿QUÉ HACE ESTO?
# - Suprime mensajes de advertencia (warnings) de Python
#
# ¿POR QUÉ?
# Algunas librerías (especialmente MDAnalysis) muestran advertencias como:
# "DeprecationWarning: función X será removida en versión Y"
# Estas advertencias:
# - Son importantes para desarrolladores de librerías
# - NO son importantes para usuarios finales (nosotros)
# - Ensucian la salida del notebook
#
# ⚠️ ADVERTENCIA IMPORTANTE:
# - Usa esto SOLO en notebooks exploratorios/educativos
# - En código de producción, SIEMPRE lee las advertencias
# - Las advertencias pueden indicar:
#   * Bugs en tu código
#   * Incompatibilidades de versiones
#   * Problemas futuros cuando actualices librerías
#
# SI TIENES PROBLEMAS:
# Comenta esta línea (ponle # adelante) para ver todas las advertencias

print("\n" + "="*70)
print("🎉 TODAS LAS LIBRERÍAS IMPORTADAS EXITOSAMENTE")
print("="*70)
print("\n📊 Resumen de versiones:")
print(f"   • OpenMM:      {mm.__version__}")
print(f"   • MDTraj:      {md.__version__}")
print(f"   • MDAnalysis:  {mda.__version__}")
print(f"   • NumPy:       {np.__version__}")
print(f"   • Pandas:      {pd.__version__}")
print("\n✅ Estás listo para comenzar la simulación")
print("="*70 + "\n")

# ¿POR QUÉ IMPRIMIR LAS VERSIONES?
# 1. Confirma que todas las importaciones funcionaron
# 2. Documenta el entorno computacional (reproducibilidad)
# 3. Ayuda a depurar si alguien tiene problemas
# 4. Algunos bugs solo existen en versiones específicas
#
# REPRODUCIBILIDAD EN CIENCIA:
# Si publicas resultados, DEBES reportar:
# - Versiones de software (OpenMM 8.0.0)
# - Sistema operativo (Ubuntu 22.04)
# - Hardware (GPU NVIDIA RTX 3080)
# Esto permite que otros repliquen tus resultados exactamente